# make dict sic

In [ ]:
from collections import defaultdict
import pickle
import os, re

In [3]:
# mlb
IN_DIR = 'data/amazon_hierarchy'
mlb_dirs = [os.path.join(IN_DIR,d) for d in os.listdir(IN_DIR) if d.startswith('mlb_cat')]
mlbs = [pickle.load(open(d,'rb')) for d in sorted(mlb_dirs)]

In [4]:
# create reverse dicts to go from class to index
class_to_index = []
for mlb in mlbs:
    class_to_index.append({cls:i for i,cls in enumerate(mlb.classes_)})

In [5]:
# parent_to_child
parent_to_child = []
for i,d in enumerate(class_to_index[:-1]):
    dd = {}
    for key,val in d.items():
        dd[val] = [v for k,v in class_to_index[i+1].items() if k.startswith(key)]
    parent_to_child.append(dd)

In [6]:
# child_to_siblings
child_to_siblings = [{}]
for i,d in enumerate(class_to_index[:-1]):
    dd = {}
    for key,val in d.items():
        siblings = [v for k,v in class_to_index[i+1].items() if k.startswith(key)]
        for sibling in siblings:
            dd[sibling]=siblings
    child_to_siblings.append(dd)
# create mask

In [ ]:
pickle.dump(child_to_siblings,open(os.path.join(IN_DIR,'child_to_siblings.pkl'),'wb'), pickle.HIGHEST_PROTOCOL)

In [ ]:
pickle.dump(parent_to_child,open(os.path.join(IN_DIR,'parent_to_child.pkl'),'wb'), pickle.HIGHEST_PROTOCOL)

# make dict amazon

In [1]:
from collections import defaultdict
import pickle
import os, re
import pandas as pd

In [2]:
# mlb
IN_DIR = 'data/amazon_hierarchy'
DF_DIR = 'data/amazon_hierarchy.pkl'
mlb_dirs = [os.path.join(IN_DIR,d) for d in os.listdir(IN_DIR) if d.startswith('mlb_cat')]
mlbs = [pickle.load(open(d,'rb')) for d in sorted(mlb_dirs)]

In [3]:
df = pd.read_pickle(DF_DIR)

In [4]:
hierarchy = set()
for h in df.hierarchy.values:
    hierarchy.add(' > '.join(h))
hierarchy = [h.split(' > ') for h in hierarchy]
len(hierarchy)

2650

In [5]:
# create reverse dicts to go from class to index
# mlb
mlb_dirs = [os.path.join(IN_DIR,d) for d in os.listdir(IN_DIR) if d.startswith('mlb_cat')]
mlbs = [pickle.load(open(d,'rb')) for d in sorted(mlb_dirs)]
class_to_index = []
for mlb in mlbs:
    class_to_index.append({cls:i for i,cls in enumerate(mlb.classes_)})

In [10]:
# parent_to_child
parent_to_child = [defaultdict(set) for _ in mlb_dirs[:-1]]
for hier in hierarchy:
    for i in range(len(hier)-1):
        key = class_to_index[i][hier[i]]
        val = class_to_index[i+1][hier[i+1]]
        parent_to_child[i][key].add(val)
parent_to_child = [{key:sorted(val) for key,val in p.items()} for p in parent_to_child]

In [11]:
# child_to_siblings
child_to_siblings = [{}]
for dd in parent_to_child:
    d = {}
    for val in dd.values():
        for v in val:
            d[v] = sorted(val)
    child_to_siblings.append(d)

In [13]:
pickle.dump(child_to_siblings,open(os.path.join(IN_DIR,'child_to_siblings.pkl'),'wb'), pickle.HIGHEST_PROTOCOL)
pickle.dump(parent_to_child,open(os.path.join(IN_DIR,'parent_to_child.pkl'),'wb'), pickle.HIGHEST_PROTOCOL)